In [4]:
import pandas as pd
import numpy as np
import joblib

from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("combined_news_dataset.csv", encoding="utf-8", on_bad_lines='skip')  # Handles bad lines
df = df.dropna(subset=["content", "label"])  # Remove rows with missing data

texts = df["content"].astype(str).tolist()
labels = df["label"].astype(int).tolist()

# Use SentenceTransformer to get embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight, fast model
X = model.encode(texts, show_progress_bar=True, batch_size=64)
y = np.array(labels)

# Scale the embeddings
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train logistic regression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Save artifacts
joblib.dump(clf, "logistic_model.pkl")
joblib.dump(scaler, "sentence_scaler.pkl")


Batches: 100%|██████████| 3266/3266 [2:06:27<00:00,  2.32s/it]  


              precision    recall  f1-score   support

           0       0.63      0.58      0.61     18573
           1       0.69      0.73      0.71     23221

    accuracy                           0.66     41794
   macro avg       0.66      0.66      0.66     41794
weighted avg       0.66      0.66      0.66     41794



['sentence_scaler.pkl']